In [45]:
# fill the area id column
import os 
import pandas as pd

def fill_area_code(df):

    df['AreaID'].dropna(inplace=True)
    cn_id = df['AreaID'][0]
    print(cn_id)
    df['AreaID'] = cn_id
    
    return df
        

In [46]:
import pandas as pd
import os

def read_and_concatenate(folder_path):
    # Lists to store DataFrames
    gen_dataframes = []
    load_dataframes = []

    # Iterate over all files in the folder
    list_files = [file for file in os.listdir(folder_path) if file!='test.csv']
    for file in list_files:
        if file.endswith('.csv'):
            print('---------------------------------')
            print(file)
            file_path = os.path.join(folder_path, file)
            

            # Read 'gen' files
            if file.startswith('gen'):
                df = pd.read_csv(file_path)
                if file == 'gen_SP_B10.csv':
                    df['AreaID'] = '10YES-REE------0'
                df = fill_area_code(df)
                df['quantity'].fillna(0, inplace=True)
                gen_dataframes.append(df)


            # Read 'load' files
            elif file.startswith('load'):
                df = pd.read_csv(file_path)
                df = fill_area_code(df)
                df['Load'].fillna(0, inplace=True)
                load_dataframes.append(df)

           
    # Concatenate DataFrames vertically
    gen_concatenated = pd.concat(gen_dataframes, axis=0, ignore_index=True)
    load_concatenated = pd.concat(load_dataframes, axis=0, ignore_index=True)

    # Combine 'gen' and 'load' DataFrames
    combined_dataframe = pd.concat([gen_concatenated, load_concatenated], axis=0, ignore_index=True)

    return combined_dataframe

# Provide the path to your folder containing the CSV files
folder_path = '../data/raw_data/'
data = read_and_concatenate(folder_path)

# Optionally, save the combined DataFrame to a new CSV file
#combined_df.to_csv('combined_dataset.csv', index=False)


---------------------------------
gen_DE_B01.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B09.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B10.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B11.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B12.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B15.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B16.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B18.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B19.csv
10Y1001A1001A83F
---------------------------------
gen_DK_B01.csv
10Y1001A1001A65H
---------------------------------
gen_DK_B16.csv
10Y1001A1001A65H
---------------------------------
gen_DK_B18.csv
10Y1001A1001A65H
---------------------------------
gen_DK_B19.csv
10Y1001A1001A65H
---------------------------------
gen_HU_B01.csv
10YHU-MAVIR----U
---------------------------------
gen_HU_B09.csv
10YHU-MAVIR----U
----------

In [49]:
data['AreaID'].unique()

array(['10Y1001A1001A83F', '10Y1001A1001A65H', '10YHU-MAVIR----U',
       '10YIT-GRTN-----B', '10YNL----------L', '10YPL-AREA-----S',
       '10YSE-1--------K', '10YES-REE------0'], dtype=object)

In [50]:
data['StartTime'] = pd.to_datetime(data['StartTime'].str.replace('\+00:00Z', '', regex=True)).dt.strftime('%Y-%m-%d %H:%M:%S')
data['EndTime'] = pd.to_datetime(data['EndTime'].str.replace('\+00:00Z', '', regex=True)).dt.strftime('%Y-%m-%d %H:%M:%S')
data['StartTime'] = pd.to_datetime(data['StartTime'])
data['EndTime'] = pd.to_datetime(data['EndTime'])

In [51]:
data['AreaID'] = data['AreaID'].replace({
    '10YHU-MAVIR----U': 'HU',
    '10YIT-GRTN-----B': 'IT',
    '10YPL-AREA-----S': 'PO',
    '10YES-REE------0': 'SP',
    # '10Y1001A1001A92E': 'UK',
    '10Y1001A1001A83F': 'DE',
    '10Y1001A1001A65H': 'DK',
    '10YSE-1--------K': 'SE',
    '10YNL----------L': 'NE'
})

In [52]:
data['AreaID'].unique()

array(['DE', 'DK', 'HU', 'IT', 'NE', 'PO', 'SE', 'SP'], dtype=object)

In [53]:
data.fillna(0, inplace=True)
data['gen/load']='load'
data['Load'] = data['Load'].fillna(0)
data.loc[data['Load']==0,'gen/load']='gen'

data['power']=data['quantity']+data['Load']
data

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,Load,gen/load,power
0,2021-12-31 23:45:00,2022-01-01 00:00:00,DE,MAW,B01,4333.0,0.0,gen,4333.0
1,2022-01-01 00:00:00,2022-01-01 00:15:00,DE,MAW,B01,4325.0,0.0,gen,4325.0
2,2022-01-01 00:15:00,2022-01-01 00:30:00,DE,MAW,B01,4319.0,0.0,gen,4319.0
3,2022-01-01 00:30:00,2022-01-01 00:45:00,DE,MAW,B01,4323.0,0.0,gen,4323.0
4,2022-01-01 00:45:00,2022-01-01 01:00:00,DE,MAW,B01,4328.0,0.0,gen,4328.0
...,...,...,...,...,...,...,...,...,...
1623568,2023-01-01 22:30:00,2023-01-01 22:45:00,SP,MAW,0,0.0,20716.0,load,20716.0
1623569,2023-01-01 22:45:00,2023-01-01 23:00:00,SP,MAW,0,0.0,20588.0,load,20588.0
1623570,2023-01-01 23:00:00,2023-01-01 23:15:00,SP,MAW,0,0.0,20116.0,load,20116.0
1623571,2023-01-01 23:15:00,2023-01-01 23:30:00,SP,MAW,0,0.0,19708.0,load,19708.0


In [54]:
# Extract date and hour
data['Date'] = data['StartTime'].dt.date
data['Hour'] = data['StartTime'].dt.hour
data

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,Load,gen/load,power,Date,Hour
0,2021-12-31 23:45:00,2022-01-01 00:00:00,DE,MAW,B01,4333.0,0.0,gen,4333.0,2021-12-31,23
1,2022-01-01 00:00:00,2022-01-01 00:15:00,DE,MAW,B01,4325.0,0.0,gen,4325.0,2022-01-01,0
2,2022-01-01 00:15:00,2022-01-01 00:30:00,DE,MAW,B01,4319.0,0.0,gen,4319.0,2022-01-01,0
3,2022-01-01 00:30:00,2022-01-01 00:45:00,DE,MAW,B01,4323.0,0.0,gen,4323.0,2022-01-01,0
4,2022-01-01 00:45:00,2022-01-01 01:00:00,DE,MAW,B01,4328.0,0.0,gen,4328.0,2022-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...
1623568,2023-01-01 22:30:00,2023-01-01 22:45:00,SP,MAW,0,0.0,20716.0,load,20716.0,2023-01-01,22
1623569,2023-01-01 22:45:00,2023-01-01 23:00:00,SP,MAW,0,0.0,20588.0,load,20588.0,2023-01-01,22
1623570,2023-01-01 23:00:00,2023-01-01 23:15:00,SP,MAW,0,0.0,20116.0,load,20116.0,2023-01-01,23
1623571,2023-01-01 23:15:00,2023-01-01 23:30:00,SP,MAW,0,0.0,19708.0,load,19708.0,2023-01-01,23


In [56]:
aggregated_data = data.groupby(['AreaID', 'gen/load', 'Date', 'Hour'])['power'].sum().reset_index()

aggregated_data['concatenated'] = aggregated_data['AreaID']  + aggregated_data['gen/load']

In [57]:
pivot = aggregated_data.pivot_table(
index=['Date', 'Hour'],
columns=['concatenated'],
values='power',
aggfunc='sum'
)
pivot

concatenated        DEgen    DEload   DKgen  DKload   HUgen   HUload   ITgen  \
Date       Hour                                                                
2021-12-31 23     39401.0   42196.0  3652.0  3314.0   285.0   4254.0  5801.0   
2022-01-01 0     157373.0  165125.0  3605.0  3218.0  1376.0  16457.0  5745.0   
           1     151747.0  160415.0  3309.0  3126.0  1526.0  15426.0  6228.0   
           2     147938.0  158035.0  3043.0  3080.0  1560.0  14781.0  6407.0   
           3     141816.0  157016.0  2852.0  3044.0  1287.0  14630.0  5760.0   
...                   ...       ...     ...     ...     ...      ...     ...   
2023-01-01 19    133596.0  177225.0  4168.0  3507.0   667.0  17873.0  3873.0   
           20    131681.0  172484.0  3975.0  3354.0   648.0  17141.0  3690.0   
           21    122205.0  166627.0  3498.0  3229.0   658.0  16259.0  3460.0   
           22    120116.0  153785.0  2879.0  3055.0   623.0  15273.0  3283.0   
           23     92697.0  110140.0     NaN     NaN   468.0  10900.0     NaN   

concatenated      ITload    NEgen   NEload   POgen   POload    SEgen   SEload  \
Date       Hour                                                                 
2021-12-31 23    21121.0   3554.0  10326.0  4641.0  14438.0  11060.0  15331.0   
2022-01-01 0     19756.0  12768.0  40706.0  4491.0  13935.0  11107.0  15331.0   
           1     18685.0  11131.0  39465.0  4436.0  13579.0  11036.0  15270.0   
           2     18124.0  11324.0  38923.0  4568.0  13397.0  10509.0  15150.0   
           3     18400.0  11290.0  38211.0  4559.0  13364.0  10770.0  15387.0   
...                  ...      ...      ...     ...      ...      ...      ...   
2023-01-01 19    24617.0   8730.0  47635.0  4968.0  15468.0  12347.0  16145.0   
           20    22944.0   7654.0  45689.0  4220.0  14698.0  11986.0  15869.0   
           21    20791.0   6577.0  43276.0  3525.0  13749.0  11227.0  15143.0   
           22    19198.0   7759.0  41269.0  3236.0  13061.0  10515.0  14697.0   
           23        NaN   5441.0  29948.0     NaN      NaN      NaN      NaN   

concatenated       SPgen   SPload  
Date       Hour                    
2021-12-31 23    11244.0  20827.0  
2022-01-01 0     10827.0  19530.0  
           1     11140.0  18383.0  
           2     11361.0  17680.0  
           3     11247.0  17396.0  
...                  ...      ...  
2023-01-01 19    76316.0  98224.0  
           20    76276.0  97620.0  
           21    72280.0  91896.0  
           22    65968.0  84324.0  
           23    49836.0  59140.0  

[8785 rows x 16 columns]

In [58]:
pivot.dropna()

concatenated        DEgen    DEload   DKgen  DKload   HUgen   HUload   ITgen  \
Date       Hour                                                                
2021-12-31 23     39401.0   42196.0  3652.0  3314.0   285.0   4254.0  5801.0   
2022-01-01 0     157373.0  165125.0  3605.0  3218.0  1376.0  16457.0  5745.0   
           1     151747.0  160415.0  3309.0  3126.0  1526.0  15426.0  6228.0   
           2     147938.0  158035.0  3043.0  3080.0  1560.0  14781.0  6407.0   
           3     141816.0  157016.0  2852.0  3044.0  1287.0  14630.0  5760.0   
...                   ...       ...     ...     ...     ...      ...     ...   
2023-01-01 18    136418.0  187476.0  4411.0  3687.0   659.0  18251.0  4946.0   
           19    133596.0  177225.0  4168.0  3507.0   667.0  17873.0  3873.0   
           20    131681.0  172484.0  3975.0  3354.0   648.0  17141.0  3690.0   
           21    122205.0  166627.0  3498.0  3229.0   658.0  16259.0  3460.0   
           22    120116.0  153785.0  2879.0  3055.0   623.0  15273.0  3283.0   

concatenated      ITload    NEgen   NEload   POgen   POload    SEgen   SEload  \
Date       Hour                                                                 
2021-12-31 23    21121.0   3554.0  10326.0  4641.0  14438.0  11060.0  15331.0   
2022-01-01 0     19756.0  12768.0  40706.0  4491.0  13935.0  11107.0  15331.0   
           1     18685.0  11131.0  39465.0  4436.0  13579.0  11036.0  15270.0   
           2     18124.0  11324.0  38923.0  4568.0  13397.0  10509.0  15150.0   
           3     18400.0  11290.0  38211.0  4559.0  13364.0  10770.0  15387.0   
...                  ...      ...      ...     ...      ...      ...      ...   
2023-01-01 18    25863.0   8357.0  49162.0  5513.0  16161.0  12715.0  16587.0   
           19    24617.0   8730.0  47635.0  4968.0  15468.0  12347.0  16145.0   
           20    22944.0   7654.0  45689.0  4220.0  14698.0  11986.0  15869.0   
           21    20791.0   6577.0  43276.0  3525.0  13749.0  11227.0  15143.0   
           22    19198.0   7759.0  41269.0  3236.0  13061.0  10515.0  14697.0   

concatenated       SPgen   SPload  
Date       Hour                    
2021-12-31 23    11244.0  20827.0  
2022-01-01 0     10827.0  19530.0  
           1     11140.0  18383.0  
           2     11361.0  17680.0  
           3     11247.0  17396.0  
...                  ...      ...  
2023-01-01 18    72728.0  95200.0  
           19    76316.0  98224.0  
           20    76276.0  97620.0  
           21    72280.0  91896.0  
           22    65968.0  84324.0  

[8783 rows x 16 columns]

In [96]:
pivot.columns

MultiIndex([(   0,  'gen'),
            ('DE',  'gen'),
            ('DE', 'load'),
            ('DK',  'gen'),
            ('DK', 'load'),
            ('HU',  'gen'),
            ('HU', 'load'),
            ('IT',  'gen'),
            ('IT', 'load'),
            ('NE',  'gen'),
            ('NE', 'load'),
            ('PO',  'gen'),
            ('PO', 'load'),
            ('SE',  'gen'),
            ('SE', 'load'),
            ('SP',  'gen'),
            ('SP', 'load'),
            ('UK',  'gen'),
            ('UK', 'load')],
           names=['AreaID', 'gen/load'])

In [97]:
pivot.columns = ['_'.join(col).strip() for col in pivot.columns.values]

TypeError: sequence item 0: expected str instance, int found

In [ ]:
pivot

AreaID                 0 10Y1001A1001A65H         10Y1001A1001A83F            \
gen/load               1                0       1                0         1   
Date       Hour                                                                
2021-12-31 23     4180.0           3314.0  3652.0          42196.0   37054.0   
2022-01-01 0     14291.0           3218.0  3605.0         165125.0  145157.0   
           1     16595.0           3126.0  3309.0         160415.0  138401.0   
           2     19011.0           3080.0  3043.0         158035.0  132617.0   
           3     18141.0           3044.0  2852.0         157016.0  127011.0   
...                  ...              ...     ...              ...       ...   
2023-01-01 19     8398.0           3507.0  4168.0         177225.0  131391.0   
           20     7269.0           3354.0  3975.0         172484.0  129744.0   
           21     6247.0           3229.0  3498.0         166627.0  121198.0   
           22    11355.0           3055.0  2879.0         153785.0  114446.0   
           23    16985.0              NaN     NaN         110140.0   81443.0   

AreaID          10Y1001A1001A92E       10YES-REE------0           \
gen/load                       0     1                0        1   
Date       Hour                                                    
2021-12-31 23              677.0   NaN          20827.0   9533.0   
2022-01-01 0              1244.0   NaN          19530.0   8943.0   
           1              1131.0   NaN          18383.0   8625.0   
           2              1091.0   NaN          17680.0   8448.0   
           3               969.0   NaN          17396.0   8290.0   
...                          ...   ...              ...      ...   
2023-01-01 19                NaN  55.0          98224.0  70192.0   
           20                NaN  45.0          97620.0  71032.0   
           21                NaN  37.0          91896.0  67136.0   
           22                NaN  41.0          84324.0  60364.0   
           23                NaN  32.0          59140.0  44148.0   

AreaID          10YHU-MAVIR----U         10YIT-GRTN-----B          \
gen/load                       0       1                0       1   
Date       Hour                                                     
2021-12-31 23             4254.0   285.0          21121.0  5698.0   
2022-01-01 0             16457.0  1376.0          19756.0  5614.0   
           1             15426.0  1526.0          18685.0  5528.0   
           2             14781.0  1560.0          18124.0  5672.0   
           3             14630.0  1287.0          18400.0  5426.0   
...                          ...     ...              ...     ...   
2023-01-01 19            17873.0   667.0          24617.0  3872.0   
           20            17141.0   648.0          22944.0  3667.0   
           21            16259.0   658.0          20791.0  3436.0   
           22            15273.0   623.0          19198.0  3264.0   
           23            10900.0   468.0              NaN     NaN   

AreaID          10YNL----------L          10YPL-AREA-----S          \
gen/load                       0        1                0       1   
Date       Hour                                                      
2021-12-31 23            10326.0   3535.0          14438.0  4641.0   
2022-01-01 0             40706.0  12708.0          13935.0  4491.0   
           1             39465.0  11097.0          13579.0  4436.0   
           2             38923.0  11282.0          13397.0  4568.0   
           3             38211.0  11245.0          13364.0  4559.0   
...                          ...      ...              ...     ...   
2023-01-01 19            47635.0   8713.0          15468.0  4968.0   
           20            45689.0   7626.0          14698.0  4220.0   
           21            43276.0   6533.0          13749.0  3525.0   
           22            41269.0   7731.0          13061.0  3236.0   
           23            29948.0   5427.0        